In [1]:
# Setup:
# 1. Uninstall previous version if already installed
%pip uninstall pyafmreader -y
# 2. Install pyafmreader from github 
# (temporary, when 1.0 releases it will be available via pip)
%pip install git+https://github.com/jlopezalo/pyafmreader.git

Found existing installation: pyafmreader x.x.x
Uninstalling pyafmreader-x.x.x:
  Successfully uninstalled pyafmreader-x.x.x
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyafmreader.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ta973ydj
  Running command git clone --filter=blob:none --quiet https://github.com/jlopezalo/pyafmreader.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ta973ydj
  Resolved https://github.com/jlopezalo/pyafmreader.git to commit a5f6da84e0a7825b980baa9d78624d341b8c2fc6
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/TkTech/fasterzip.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-t4itruep/fasterzip_023af947f72b49a99b65756c4f52c0bd
  Running command git clone --filter=blob:none --quiet https://github.com/TkTech/fasterzip.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-t4itru

In [2]:
# 1. Import pyafmreader loadfile and matplotlib
from pyafmreader import loadfile
import matplotlib.pyplot as plt

# Get documentation about loadfile function
help(loadfile)

Help on function loadfile in module pyafmreader.pyafmreader:

loadfile(filepath)
    Load AFM file. 
    
    Supported formats:
        - JPK --> .jpk-force, .jpk-force-map, .jpk-qi-data
        - JPK Thermal --> .tnd
        - NANOSCOPE --> .spm, .pfc
        - UFF --> .uff
    
            Parameters:
                    filepath (str): Path to the file.
            
            Returns:
                    If JPK, NANOSCOPE OR UFF:
                        UFF (uff.UFF): Universal File Format object containing loaded data.
                    If JPK Thermal:
                        Amplitude (m^2/V) (np.array),
                        Frequencies (Hz) (np.array),
                        Fit-Data (m^2/V) (np.array),
                        Parameters (dict)



In [3]:
# 2. Load one of the test files
# NANOSC_PFC_PATH = '../tests/testfiles/08171528.0_00001.pfc'
NANOSC_PFC_PATH = '/Users/javierlopez/Desktop/CellHesion_data_set/force-save-2022.08.02-16.02.01.977.jpk-force'

NANOSC_PFC_FILE = loadfile(NANOSC_PFC_PATH)

type(NANOSC_PFC_FILE)

pyafmreader.uff.UFF

In [4]:
# 3. Get file metadata
metadata = NANOSC_PFC_FILE.filemetadata

for key, item in metadata.items():
    print(f"{key} : {item}\n") 

file_path : /Users/javierlopez/Desktop/CellHesion_data_set/force-save-2022.08.02-16.02.01.977.jpk-force

Entry_filename : force-save-2022.08.02-16.02.01.977.jpk-force

file_size_bytes : 17129

file_type : .jpk-force

UFF_code : _1_2_3_4_5

Entry_UFF_version : 0

file_id : 16.02.01.977

force_volume : 0

Experimental_instrument : JPK01385-CellHesion-200

JPK_file_format_version : 2.0

JPK_software_version : 5.0.138

retracted_delay : 0.0

extended_delay : 0.0

Entry_date : None

scan_angle : 0.0

num_x_pixels : 1

num_y_pixels : 1

scan_size_x : 0.0

scan_size_y : 0.0

z_closed_loop : false

Recording_Z_close_loop_on : Off

Entry_tot_nb_curve : 1

extend_pause_duration : 0.0

relative_z_start : 0.0

relative_z_end : 0.0

relative_ramp_size : 0.0

force_setpoint : 7.5945945178241026

nbr_channels : 6

defl_sens_nmbyV : 29.718593330029282

spring_const_Nbym : 0.10633553027241671

channel_properties : {'cellhesion-height': {'channel_id': 0, 'encoder_type': 'signedshort', 'encoder_offet_key

In [7]:
# 4. Get force volume piezo image and plot it
# piezoimg = NANOSC_PFC_FILE.getpiezoimg()

# plt.figure(figsize=(10,10))
# plt.imshow(piezoimg[:, :, 0], cmap='afmhot', origin='lower')
# plt.show()

In [ ]:
height_channel_key = file_metadata['height_channel_key']
print()

In [6]:
# 5. Get force curve at index 0
FC = NANOSC_PFC_FILE.getcurve(0)

type(FC)

KeyError: 'absolute_defined'

In [ ]:
help(FC.preprocess_force_curve)

In [ ]:
# 6. Preprocess curve with the deflection sens in the header
defl_sens = metadata['defl_sens_nmbyV'] / 1e09 # nm/V --> m/V
FC.preprocess_force_curve(defl_sens, metadata['height_channel_key'])
plt.figure(figsize=(10,5))
for segid, segment in FC.get_segments():
    plt.plot(segment.zheight, segment.vdeflection, label=f'Segment: {segid}')
plt.xlabel('zheight [m]')
plt.ylabel('vdeflection [m]')
plt.legend()
plt.show()

In [ ]:
# 7. Get Force vs Indentation curve
poc = [0.5 * 1e-8, 0.0] # in nm
spring_k = metadata['spring_const_Nbym']
FC.get_force_vs_indentation(poc, spring_k)
plt.figure(figsize=(10,5))
for segid, segment in FC.get_segments():
    plt.plot(segment.indentation, segment.force, label=f'Segment: {segid}')
plt.axhline(y=0, color='k', linestyle='--')
plt.axvline(x=0, color='k', linestyle='--')
plt.xlabel('Indentation [m]')
plt.ylabel('Force [N]')
plt.legend()
plt.show()

In [ ]:
# 8. Get segment 0
FC_segments = FC.get_segments()
_, segment_0 = FC_segments[0]
type(segment_0)

In [ ]:
# 9. Plot only segment 0
plt.figure(figsize=(10,5))
plt.plot(segment_0.zheight, segment_0.vdeflection, label=f'Segment: 0')
plt.xlabel('zheight [nm]')
plt.ylabel('vdeflection [nm]')
plt.legend()
plt.show()

In [ ]:
# 9. Get force vs indentation for segment 0
plt.figure(figsize=(10,5))
plt.plot(segment_0.indentation, segment_0.force, label=f'Segment: 0')
plt.xlabel('zheight [nm]')
plt.ylabel('vdeflection [nm]')
plt.legend()
plt.show()